In [26]:
import os
import json
from dotenv import load_dotenv
from supabase import create_client
from langchain_openai import ChatOpenAI
from langchain_core.tools import tool
from langchain_core.messages import HumanMessage, SystemMessage
from langgraph.graph import StateGraph, MessagesState, END, START
from langgraph.prebuilt.tool_node import ToolNode, tools_condition
from langgraph.checkpoint.postgres.aio import AsyncPostgresSaver

In [27]:
load_dotenv()
SUPABASE_URL = os.getenv("SUPABASE_URL")
SUPABASE_KEY = os.getenv("SUPABASE_KEY")
SUPABASE_CONNECTION_STRING = os.getenv("SUPABASE_CONNECTION_STRING")

In [28]:
supabase = create_client(SUPABASE_URL, SUPABASE_KEY)
memory = AsyncPostgresSaver.from_conn_string(SUPABASE_CONNECTION_STRING)

In [29]:
@tool
def update_active_events_tool(active_events: list):
    """
    Updates the active events list saves it to Supabase.
    """
    try:
        supabase.table("active_events").upsert({"id": 1, "events": active_events}).execute()
    except Exception as e:
        return f"Error parsing JSON: {e}"

In [42]:
llm = ChatOpenAI(model="gpt-4o", temperature=0)
llm_with_tools = llm.bind_tools([update_active_events_tool], return_direct=True)

In [33]:
MODEL_SYSTEM_MESSAGE = """You are an AI assistant that helps users manage their home security system.
You can update the user's active monitoring events based on their requests.
You have access to a tool called update_active_events_tool that allows you to modify the active events list.
You can use this tool to add, remove, or modify events as per the user's instructions.
You have read-only knowledge of the user's cameras and their descriptions to understand event requests.
Camera descriptions (read-only, do not change):
{cameras}
Current active events (editable):
{active_events}
INSTRUCTIONS:
1. Identify any updates to the active events list:
    - Add new events explicitly mentioned by the user.
    - Remove events that the user explicitly requests to stop monitoring.
    - Modify existing events if the user provides new details.  
    """


In [34]:
class AgentState(MessagesState):
    active_events: list
    cameras: list

In [48]:
def chat_node(state: AgentState):
    system_message = SystemMessage(
        content=MODEL_SYSTEM_MESSAGE.format(
            cameras=json.dumps(state["cameras"], indent=2),
            active_events=json.dumps(state["active_events"], indent=2)
        )
    )    
    messages = [system_message] + state["chat_history"]
    reply = llm_with_tools.invoke(messages)
    return {"chat_history": state["chat_history"] + [reply]}


In [50]:
def confirmation_node(state: AgentState):
    updated_events = json.dumps(state["active_events"], indent=2)
    message = f"I've updated your active events list. Here’s the current list:\n{updated_events}"
    return {"chat_history": state["chat_history"] + [HumanMessage(content=message)]}

In [ ]:
graph_builder = StateGraph(AgentState)

graph_builder.add_node("chat", chat_node)
graph_builder.add_node("update_events", ToolNode(tools=[update_active_events_tool]))
graph_builder.add_node("confirmation", confirmation_node)

graph_builder.add_edge(START, "chat")
graph_builder.add_conditional_edges("chat", "update_events", tools_condition(llm_with_tools))
graph_builder.add_edge("update_events", "confirmation")
graph_builder.add_edge("confirmation", END)


ValueError: No messages found in input state to tool_edge: bound=ChatOpenAI(client=<openai.resources.chat.completions.completions.Completions object at 0x760598a8e030>, async_client=<openai.resources.chat.completions.completions.AsyncCompletions object at 0x760598a8e360>, root_client=<openai.OpenAI object at 0x760598a8de10>, root_async_client=<openai.AsyncOpenAI object at 0x760598a8e140>, model_name='gpt-4o', temperature=0.0, model_kwargs={}, openai_api_key=SecretStr('**********')) kwargs={'tools': [{'type': 'function', 'function': {'name': 'update_active_events_tool', 'description': 'Updates the active events list saves it to Supabase.', 'parameters': {'properties': {'active_events': {'items': {}, 'type': 'array'}}, 'required': ['active_events'], 'type': 'object'}}}], 'return_direct': True} config={} config_factories=[]

In [ ]:
cameras = [
    {"id": "cam1", "description": "Front door camera"},
    {"id": "cam2", "description": "Backyard camera"},
    {"id": "cam3", "description": "Garage camera"},
]
active_events = [
    {"id": "event1", "description": "Motion detected at front door"},
    {"id": "event2", "description": "Garage door opened"},
]
agent.invoke({
    "chat_history": HumanMessage(content ="User: Monitor the backyard for presence of a car."),
    "active_events": active_events,
    "cameras": cameras
})

{'chat_history': "User: Monitor the backyard for presence of a car.\nAssistant: To monitor your backyard for the presence of a car, you can consider the following options:\n\n1. **Security Cameras**: Install security cameras with motion detection features. These cameras can send alerts to your phone or email when they detect movement, allowing you to check if a car is present.\n\n2. **Smart Sensors**: Use smart motion sensors that can detect large objects like cars. These sensors can be integrated with a smart home system to notify you of any activity.\n\n3. **Drones**: If you have a large backyard, consider using a drone for periodic aerial surveillance. Some drones can be programmed to patrol specific areas and send live video feeds to your device.\n\n4. **Lighting**: Install motion-activated lights in your backyard. While this won't directly notify you of a car's presence, it can deter unauthorized vehicles and alert you to movement when the lights turn on.\n\n5. **Regular Checks**: